In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np

In [2]:
# Path to save logs into.
logs_path = '/tmp/tensorflow_logs/example/'

# MNIST dataset parameters.
num_classes = 10 # total classes (0-9 digits).
num_features = 784 # data features (img shape: 28*28).

# Training parameters.
learning_rate = 0.001
training_steps = 3000
batch_size = 256
display_step = 100

# Network parameters.
n_hidden_1 = 128 # 1st layer number of neurons.
n_hidden_2 = 256 # 2nd layer number of neurons.

# Prepare MNIST data.
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [18]:
# Store layers weight & bias

# A random value generator to initialize weights.
random_normal = tf.initializers.RandomNormal()

weights = {
    'h1_weights': tf.Variable(random_normal([num_features, n_hidden_1]), name='h1_weights'),
    'h2_weights': tf.Variable(random_normal([n_hidden_1, n_hidden_2]), name='h2_weights'),
    'logits_weights': tf.Variable(random_normal([n_hidden_2, num_classes]), name='logits_weights')
}
biases = {
    'h1_bias': tf.Variable(tf.zeros([n_hidden_1]), name='h1_bias'),
    'h2_bias': tf.Variable(tf.zeros([n_hidden_2]), name='h2_bias'),
    'logits_bias': tf.Variable(tf.zeros([num_classes]), name='logits_bias')
}

In [19]:
# Construct model and encapsulating all ops into scopes, making
# Tensorboard's Graph visualization more convenient.

# The computation graph to be traced.
@tf.function
def neural_net(x):
    with tf.name_scope('Model'):
        with tf.name_scope('HiddenLayer1'):
        # Hidden fully connected layer with 128 neurons.
            layer_1 = tf.add(tf.matmul(x, weights['h1_weights']), biases['h1_bias'])
            # Apply sigmoid to layer_1 output for non-linearity.
            layer_1 = tf.nn.sigmoid(layer_1)
        with tf.name_scope('HiddenLayer2'):
            # Hidden fully connected layer with 256 neurons.
            layer_2 = tf.add(tf.matmul(layer_1, weights['h2_weights']), biases['h2_bias'])
            # Apply sigmoid to layer_2 output for non-linearity.
            layer_2 = tf.nn.sigmoid(layer_2)
        with tf.name_scope('LogitsLayer'):
            # Output fully connected layer with a neuron for each class.
            out_layer = tf.matmul(layer_2, weights['logits_weights']) + biases['logits_bias']
            # Apply softmax to normalize the logits to a probability distribution.
            out_layer = tf.nn.softmax(out_layer)
    return out_layer

In [20]:
# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    with tf.name_scope('CrossEntropyLoss'):
        # Encode label to a one hot vector.
        y_true = tf.one_hot(y_true, depth=num_classes)
        # Clip prediction values to avoid log(0) error.
        y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
        # Compute cross-entropy.
        return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# Accuracy metric.
def accuracy(y_pred, y_true):
    with tf.name_scope('Accuracy'):
        # Predicted class is the index of highest score in prediction vector (i.e. argmax).
        correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
        return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Stochastic gradient descent optimizer.
with tf.name_scope('Optimizer'):
    optimizer = tf.optimizers.SGD(learning_rate)

In [21]:
train_summary_writer = tf.summary.create_file_writer("/tmp/log/train")
test_summary_writer = tf.summary.create_file_writer("/tmp/log/test")

In [22]:
# Optimization process. 
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = neural_net(x)
        loss = cross_entropy(pred, y)
        acc = accuracy(pred, y)
        
    # Variables to update, i.e. trainable variables.
    trainable_variables = list(weights.values())# + biases.values()

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    
    # Update weights/biases following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    
    return loss, acc

In [23]:
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    loss, acc = run_optimization(batch_x, batch_y)
    
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', loss, step=step)
        tf.summary.scalar('acc', acc, step=step)
    print('step', step, 'loss', loss, 'acc', acc)
        
    if step %100 ==0:
        pred = neural_net(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        with test_summary_writer.as_default():
            tf.summary.scalar('loss', loss, step=step)
            tf.summary.scalar('acc', acc, step=step)
        print('test_step', step, 'loss', loss, 'acc', acc)



step 1 loss tf.Tensor(598.01355, shape=(), dtype=float32) acc tf.Tensor(0.09765625, shape=(), dtype=float32)
step 2 loss tf.Tensor(596.3518, shape=(), dtype=float32) acc tf.Tensor(0.0859375, shape=(), dtype=float32)
step 3 loss tf.Tensor(616.85046, shape=(), dtype=float32) acc tf.Tensor(0.125, shape=(), dtype=float32)
step 4 loss tf.Tensor(608.31824, shape=(), dtype=float32) acc tf.Tensor(0.10546875, shape=(), dtype=float32)
step 5 loss tf.Tensor(619.053, shape=(), dtype=float32) acc tf.Tensor(0.0859375, shape=(), dtype=float32)
step 6 loss tf.Tensor(622.19434, shape=(), dtype=float32) acc tf.Tensor(0.08984375, shape=(), dtype=float32)
step 7 loss tf.Tensor(609.624, shape=(), dtype=float32) acc tf.Tensor(0.1640625, shape=(), dtype=float32)
step 8 loss tf.Tensor(613.35986, shape=(), dtype=float32) acc tf.Tensor(0.078125, shape=(), dtype=float32)
step 9 loss tf.Tensor(613.22253, shape=(), dtype=float32) acc tf.Tensor(0.10546875, shape=(), dtype=float32)
step 10 loss tf.Tensor(593.58136, 

step 80 loss tf.Tensor(579.41766, shape=(), dtype=float32) acc tf.Tensor(0.109375, shape=(), dtype=float32)
step 81 loss tf.Tensor(574.6957, shape=(), dtype=float32) acc tf.Tensor(0.15234375, shape=(), dtype=float32)
step 82 loss tf.Tensor(583.3376, shape=(), dtype=float32) acc tf.Tensor(0.1015625, shape=(), dtype=float32)
step 83 loss tf.Tensor(579.77637, shape=(), dtype=float32) acc tf.Tensor(0.09765625, shape=(), dtype=float32)
step 84 loss tf.Tensor(574.11865, shape=(), dtype=float32) acc tf.Tensor(0.11328125, shape=(), dtype=float32)
step 85 loss tf.Tensor(577.8387, shape=(), dtype=float32) acc tf.Tensor(0.14453125, shape=(), dtype=float32)
step 86 loss tf.Tensor(571.4419, shape=(), dtype=float32) acc tf.Tensor(0.15625, shape=(), dtype=float32)
step 87 loss tf.Tensor(570.115, shape=(), dtype=float32) acc tf.Tensor(0.14453125, shape=(), dtype=float32)
step 88 loss tf.Tensor(573.51025, shape=(), dtype=float32) acc tf.Tensor(0.1171875, shape=(), dtype=float32)
step 89 loss tf.Tensor(

step 160 loss tf.Tensor(512.28815, shape=(), dtype=float32) acc tf.Tensor(0.43359375, shape=(), dtype=float32)
step 161 loss tf.Tensor(501.41467, shape=(), dtype=float32) acc tf.Tensor(0.27734375, shape=(), dtype=float32)
step 162 loss tf.Tensor(502.90808, shape=(), dtype=float32) acc tf.Tensor(0.35546875, shape=(), dtype=float32)
step 163 loss tf.Tensor(502.9192, shape=(), dtype=float32) acc tf.Tensor(0.40234375, shape=(), dtype=float32)
step 164 loss tf.Tensor(486.68063, shape=(), dtype=float32) acc tf.Tensor(0.41796875, shape=(), dtype=float32)
step 165 loss tf.Tensor(480.41644, shape=(), dtype=float32) acc tf.Tensor(0.49609375, shape=(), dtype=float32)
step 166 loss tf.Tensor(487.44785, shape=(), dtype=float32) acc tf.Tensor(0.375, shape=(), dtype=float32)
step 167 loss tf.Tensor(487.88406, shape=(), dtype=float32) acc tf.Tensor(0.47265625, shape=(), dtype=float32)
step 168 loss tf.Tensor(474.3738, shape=(), dtype=float32) acc tf.Tensor(0.421875, shape=(), dtype=float32)
step 169 l

step 237 loss tf.Tensor(316.09106, shape=(), dtype=float32) acc tf.Tensor(0.6484375, shape=(), dtype=float32)
step 238 loss tf.Tensor(305.04907, shape=(), dtype=float32) acc tf.Tensor(0.54296875, shape=(), dtype=float32)
step 239 loss tf.Tensor(315.92084, shape=(), dtype=float32) acc tf.Tensor(0.59375, shape=(), dtype=float32)
step 240 loss tf.Tensor(324.91196, shape=(), dtype=float32) acc tf.Tensor(0.5625, shape=(), dtype=float32)
step 241 loss tf.Tensor(314.3478, shape=(), dtype=float32) acc tf.Tensor(0.60546875, shape=(), dtype=float32)
step 242 loss tf.Tensor(302.80548, shape=(), dtype=float32) acc tf.Tensor(0.68359375, shape=(), dtype=float32)
step 243 loss tf.Tensor(329.0031, shape=(), dtype=float32) acc tf.Tensor(0.58203125, shape=(), dtype=float32)
step 244 loss tf.Tensor(313.71838, shape=(), dtype=float32) acc tf.Tensor(0.63671875, shape=(), dtype=float32)
step 245 loss tf.Tensor(305.74292, shape=(), dtype=float32) acc tf.Tensor(0.6328125, shape=(), dtype=float32)
step 246 los

step 313 loss tf.Tensor(237.0282, shape=(), dtype=float32) acc tf.Tensor(0.703125, shape=(), dtype=float32)
step 314 loss tf.Tensor(228.65402, shape=(), dtype=float32) acc tf.Tensor(0.71875, shape=(), dtype=float32)
step 315 loss tf.Tensor(249.17763, shape=(), dtype=float32) acc tf.Tensor(0.69921875, shape=(), dtype=float32)
step 316 loss tf.Tensor(241.13818, shape=(), dtype=float32) acc tf.Tensor(0.68359375, shape=(), dtype=float32)
step 317 loss tf.Tensor(243.08539, shape=(), dtype=float32) acc tf.Tensor(0.74609375, shape=(), dtype=float32)
step 318 loss tf.Tensor(226.38364, shape=(), dtype=float32) acc tf.Tensor(0.6796875, shape=(), dtype=float32)
step 319 loss tf.Tensor(210.80396, shape=(), dtype=float32) acc tf.Tensor(0.76171875, shape=(), dtype=float32)
step 320 loss tf.Tensor(231.82892, shape=(), dtype=float32) acc tf.Tensor(0.703125, shape=(), dtype=float32)
step 321 loss tf.Tensor(216.48209, shape=(), dtype=float32) acc tf.Tensor(0.7578125, shape=(), dtype=float32)
step 322 lo

...

step 2629 loss tf.Tensor(101.012146, shape=(), dtype=float32) acc tf.Tensor(0.890625, shape=(), dtype=float32)
step 2630 loss tf.Tensor(52.268467, shape=(), dtype=float32) acc tf.Tensor(0.9375, shape=(), dtype=float32)
step 2631 loss tf.Tensor(80.26291, shape=(), dtype=float32) acc tf.Tensor(0.91796875, shape=(), dtype=float32)
step 2632 loss tf.Tensor(71.876564, shape=(), dtype=float32) acc tf.Tensor(0.91796875, shape=(), dtype=float32)
step 2633 loss tf.Tensor(76.76595, shape=(), dtype=float32) acc tf.Tensor(0.91015625, shape=(), dtype=float32)
step 2634 loss tf.Tensor(61.820545, shape=(), dtype=float32) acc tf.Tensor(0.9296875, shape=(), dtype=float32)
step 2635 loss tf.Tensor(36.557724, shape=(), dtype=float32) acc tf.Tensor(0.96875, shape=(), dtype=float32)
step 2636 loss tf.Tensor(57.439682, shape=(), dtype=float32) acc tf.Tensor(0.9296875, shape=(), dtype=float32)
step 2637 loss tf.Tensor(80.8439, shape=(), dtype=float32) acc tf.Tensor(0.89453125, shape=(), dtype=float32)
step 2

step 2714 loss tf.Tensor(62.044266, shape=(), dtype=float32) acc tf.Tensor(0.92578125, shape=(), dtype=float32)
step 2715 loss tf.Tensor(78.11125, shape=(), dtype=float32) acc tf.Tensor(0.8984375, shape=(), dtype=float32)
step 2716 loss tf.Tensor(64.76483, shape=(), dtype=float32) acc tf.Tensor(0.9140625, shape=(), dtype=float32)
step 2717 loss tf.Tensor(55.72995, shape=(), dtype=float32) acc tf.Tensor(0.9375, shape=(), dtype=float32)
step 2718 loss tf.Tensor(56.35105, shape=(), dtype=float32) acc tf.Tensor(0.94140625, shape=(), dtype=float32)
step 2719 loss tf.Tensor(61.595642, shape=(), dtype=float32) acc tf.Tensor(0.91015625, shape=(), dtype=float32)
step 2720 loss tf.Tensor(62.34784, shape=(), dtype=float32) acc tf.Tensor(0.921875, shape=(), dtype=float32)
step 2721 loss tf.Tensor(38.438408, shape=(), dtype=float32) acc tf.Tensor(0.95703125, shape=(), dtype=float32)
step 2722 loss tf.Tensor(48.88056, shape=(), dtype=float32) acc tf.Tensor(0.94140625, shape=(), dtype=float32)
step 2

step 2793 loss tf.Tensor(43.204113, shape=(), dtype=float32) acc tf.Tensor(0.94140625, shape=(), dtype=float32)
step 2794 loss tf.Tensor(65.71626, shape=(), dtype=float32) acc tf.Tensor(0.93359375, shape=(), dtype=float32)
step 2795 loss tf.Tensor(57.04993, shape=(), dtype=float32) acc tf.Tensor(0.921875, shape=(), dtype=float32)
step 2796 loss tf.Tensor(50.269043, shape=(), dtype=float32) acc tf.Tensor(0.95703125, shape=(), dtype=float32)
step 2797 loss tf.Tensor(52.996048, shape=(), dtype=float32) acc tf.Tensor(0.94921875, shape=(), dtype=float32)
step 2798 loss tf.Tensor(67.96576, shape=(), dtype=float32) acc tf.Tensor(0.92578125, shape=(), dtype=float32)
step 2799 loss tf.Tensor(57.048706, shape=(), dtype=float32) acc tf.Tensor(0.93359375, shape=(), dtype=float32)
step 2800 loss tf.Tensor(56.01099, shape=(), dtype=float32) acc tf.Tensor(0.93359375, shape=(), dtype=float32)
test_step 2800 loss tf.Tensor(50.732346, shape=(), dtype=float32) acc tf.Tensor(0.94140625, shape=(), dtype=fl

step 2872 loss tf.Tensor(57.938114, shape=(), dtype=float32) acc tf.Tensor(0.93359375, shape=(), dtype=float32)
step 2873 loss tf.Tensor(61.05152, shape=(), dtype=float32) acc tf.Tensor(0.93359375, shape=(), dtype=float32)
step 2874 loss tf.Tensor(46.14173, shape=(), dtype=float32) acc tf.Tensor(0.95703125, shape=(), dtype=float32)
step 2875 loss tf.Tensor(29.69076, shape=(), dtype=float32) acc tf.Tensor(0.98046875, shape=(), dtype=float32)
step 2876 loss tf.Tensor(71.118774, shape=(), dtype=float32) acc tf.Tensor(0.91796875, shape=(), dtype=float32)
step 2877 loss tf.Tensor(52.13933, shape=(), dtype=float32) acc tf.Tensor(0.94921875, shape=(), dtype=float32)
step 2878 loss tf.Tensor(53.908306, shape=(), dtype=float32) acc tf.Tensor(0.92578125, shape=(), dtype=float32)
step 2879 loss tf.Tensor(60.02311, shape=(), dtype=float32) acc tf.Tensor(0.9375, shape=(), dtype=float32)
step 2880 loss tf.Tensor(75.541595, shape=(), dtype=float32) acc tf.Tensor(0.91796875, shape=(), dtype=float32)
s

step 2946 loss tf.Tensor(45.108562, shape=(), dtype=float32) acc tf.Tensor(0.9375, shape=(), dtype=float32)
step 2947 loss tf.Tensor(54.169006, shape=(), dtype=float32) acc tf.Tensor(0.93359375, shape=(), dtype=float32)
step 2948 loss tf.Tensor(48.5464, shape=(), dtype=float32) acc tf.Tensor(0.9453125, shape=(), dtype=float32)
step 2949 loss tf.Tensor(45.258297, shape=(), dtype=float32) acc tf.Tensor(0.9375, shape=(), dtype=float32)
step 2950 loss tf.Tensor(54.303116, shape=(), dtype=float32) acc tf.Tensor(0.93359375, shape=(), dtype=float32)
step 2951 loss tf.Tensor(58.65884, shape=(), dtype=float32) acc tf.Tensor(0.9296875, shape=(), dtype=float32)
step 2952 loss tf.Tensor(55.768757, shape=(), dtype=float32) acc tf.Tensor(0.9375, shape=(), dtype=float32)
step 2953 loss tf.Tensor(74.83389, shape=(), dtype=float32) acc tf.Tensor(0.8984375, shape=(), dtype=float32)
step 2954 loss tf.Tensor(50.8431, shape=(), dtype=float32) acc tf.Tensor(0.94921875, shape=(), dtype=float32)
step 2955 los